In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="png", remove_last=False)
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,Architecture
0,model_2_9_16,0.998570,0.747512,0.995817,0.997521,0.997362,0.006000,1.688391,0.013223,0.019823,0.016523,"Hidden Size=[5, 6], regularizer=0.02, learning..."
1,model_2_9_17,0.998569,0.747419,0.995789,0.997421,0.997291,0.006006,1.689011,0.013308,0.020624,0.016966,"Hidden Size=[5, 6], regularizer=0.02, learning..."
2,model_2_9_15,0.998568,0.747612,0.995846,0.997631,0.997439,0.006012,1.687719,0.013130,0.018947,0.016039,"Hidden Size=[5, 6], regularizer=0.02, learning..."
3,model_2_9_18,0.998564,0.747333,0.995764,0.997330,0.997226,0.006026,1.689583,0.013388,0.021354,0.017371,"Hidden Size=[5, 6], regularizer=0.02, learning..."
4,model_2_9_14,0.998559,0.747721,0.995877,0.997751,0.997523,0.006049,1.686994,0.013031,0.017990,0.015511,"Hidden Size=[5, 6], regularizer=0.02, learning..."
5,model_2_9_19,0.998557,0.747255,0.995741,0.997247,0.997167,0.006055,1.690107,0.013460,0.022019,0.017740,"Hidden Size=[5, 6], regularizer=0.02, learning..."
6,model_2_9_20,0.998549,0.747183,0.995720,0.997171,0.997114,0.006091,1.690588,0.013527,0.022623,0.018075,"Hidden Size=[5, 6], regularizer=0.02, learning..."
7,model_2_9_13,0.998542,0.747837,0.995911,0.997881,0.997615,0.006120,1.686216,0.012926,0.016949,0.014937,"Hidden Size=[5, 6], regularizer=0.02, learning..."
8,model_2_9_21,0.998539,0.747117,0.995701,0.997103,0.997065,0.006130,1.691027,0.013589,0.023172,0.018380,"Hidden Size=[5, 6], regularizer=0.02, learning..."
9,model_2_9_22,0.998529,0.747057,0.995683,0.997040,0.997021,0.006171,1.691428,0.013645,0.023669,0.018657,"Hidden Size=[5, 6], regularizer=0.02, learning..."
